In [1]:
import numpy as np
import pandas as pd

In [2]:

df_raw = pd.read_csv('data/pilot/raw/20_01_2025/tom_dictator_v2_2025-01-20.csv')
                     
# Replace full stop with underscore in df_raw column names
df_raw.columns = df_raw.columns.str.replace('.', '_')

sessions = [
    'tgvqauld',
    'ta176utg'
]

# Only keep rows where session_id is in sessions and where participant_label is not NaN
df_full = df_raw[df_raw['session_code'].isin(sessions) & df_raw['participant_label'].notna() & (df_raw['participant__current_page_name'] == 'End')]
# Insert 
columns = ['participant_code', 'participant_label', 'session_code',
       
       'player_block_type', 'player_block_idx', 'player_idx_in_block', 'player_trust_cond', 'player_trust_label',
       'player_intention', 'player_interest', 
       'player_certainty', 'player_certainty_val', 'player_rigidity', 'player_rigidity_val',
       'player_k_lvl', 'player_trust_predicted', 'player_alpha_prior',
       'player_alpha_prior_entropy', 'player_alpha', 'player_alpha_entropy',
       'player_beta_prior', 'player_beta_prior_entropy', 'player_beta',
       'player_beta_entropy', 'player_beta_social_prior',
       'player_beta_social_prior_entropy', 'player_beta_social',
       'player_beta_social_entropy', 'player_path_taken',
       'player_perceived_certainty_certain',
       'player_perceived_certainty_uncertain',
       'player_perceived_certainty_immutable', 'player_attention',
       'player_attention_passed', 'player_attention_correct',

       'player_focus_work_high', 'player_focus_work_medium', 'player_focus_work_low',
       'player_strategy_high', 'player_strategy_low', 'player_strategy_medium',

       'subsession_round_number',
       'participant__current_app_name',
       'participant__current_page_name', 
       'player_payoff',
]


df = df_full[columns]

# If player is in column name, remove player_ from column name
df.columns = df.columns.str.replace('player_', '')


df_long = df[(df.block_type == 'test')][['participant_code', 'intention', 'certainty', 'rigidity', 'intention', 'trust_cond', 'path_taken']]

df_long['absent'] = df_long.trust_cond.map(lambda x: x == 'none')

# Get dummies for path taken and concat with df_long
path_taken_dummies = pd.get_dummies(df_long['path_taken'], prefix='', prefix_sep='')
df_long = pd.concat([df_long, path_taken_dummies], axis=1)

df_long['delivered'] = df_long['A'] + df_long['B'] > 0
df_long['path_hidden'] = df_long['B'] + df_long['C'] > 0

# Create path_idx column
path_dict = {
    'A': 1,
    'B': 2,
    'C': 3,
    'D': 4,
}
df['path_idx'] = df['path_taken'].map(lambda x: path_dict[x])
df_long['path_length'] = df_long['path_taken'].map(lambda x: path_dict[x])

# Create certainty_lvl column
certainties_dict = {
    'uncertain': 1,
    'certain': 2,
    'immutable': 3,
}
df_long['certainty_lvl'] = df_long['certainty'].map(lambda x: certainties_dict[x])
df['certainty_lvl'] = df['certainty'].map(lambda x: certainties_dict[x])

# Create trust_lvl column
trust_dict = {
    'none': 4,
    'low': 1,
    'medium': 2,
    'high': 3,
}
df_long['trust_lvl'] = df_long['trust_cond'].map(lambda x: trust_dict[x])
df['trust_lvl'] = df['trust_cond'].map(lambda x: trust_dict[x])

# Assign unique integer to each participant and make sure it is the same for both df and df_long and recoverable
df_long['part_idx'] = pd.Categorical(df_long['participant_code']).codes
df['part_idx'] = pd.Categorical(df['participant_code']).codes



paths_colors = {
    'A': np.array([238, 154, 86])/255,
    'B': np.array([183, 117, 63])/255,
    'C': np.array([50, 114, 169])/255,
    'D': np.array([88, 182, 225])/255,
}

paths_colors_next = {
    'A': np.array([183, 117, 63])/255,
    'B': np.array([50, 114, 169])/255,
    'C': np.array([88, 182, 225])/255,
}

/var/folders/hh/c12d0kb97ln3mqms4k45kqwm0000gn/T/ipykernel_40583/2952042552.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['path_idx'] = df['path_taken'].map(lambda x: path_dict[x])
/var/folders/hh/c12d0kb97ln3mqms4k45kqwm0000gn/T/ipykernel_40583/2952042552.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['certainty_lvl'] = df['certainty'].map(lambda x: certainties_dict[x])
/var/folders/hh/c12d0kb97ln3mqms4k45kqwm0000gn/T/ipykernel_40583/2952042552.py:84: SettingWithCopyWarning: 
A value is 

In [3]:
df

,participant_code,participant_label,session_code,block_type,block_idx,idx_in_block,trust_cond,trust_label,intention,interest,...,strategy_low,strategy_medium,subsession_round_number,participant__current_app_name,participant__current_page_name,payoff,path_idx,certainty_lvl,trust_lvl,part_idx
66,ivf2doqd,5c4c57609446be00017e6746,tgvqauld,explore,0,0,medium,medium,none,interested,...,NaN,NaN,1,generic_demographics,End,0.0,3,1,2,54
68,hh2ezh9j,650af5c1e51d63383d770aea,tgvqauld,explore,0,0,medium,medium,none,interested,...,NaN,NaN,1,generic_demographics,End,0.0,2,1,2,48
69,g3l64f2k,60fd3585f5bbc85cfc3634cf,tgvqauld,explore,0,0,medium,medium,none,interested,...,NaN,NaN,1,generic_demographics,End,0.0,2,1,2,42
70,qwthoytg,6710229227bead028cf87729,tgvqauld,explore,0,0,medium,medium,none,interested,...,NaN,NaN,1,generic_demographics,End,0.0,1,1,2,74
72,81qc8udv,667ad8ffae84b34e30e27b72,tgvqauld,explore,0,0,medium,medium,none,interested,...,NaN,NaN,1,generic_demographics,End,0.0,2,1,2,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11094,fc2ylfo4,678738d9b8adc7b432041641,ta176utg,test,20,0,none,none,none,interested,...,I always tried to balance being on time vs ple...,I always tried to balance being on time vs ple...,65,generic_demographics,End,3.0,4,1,4,41
11095,96sj342h,675de3fe4f6075a2d3a3a798,ta176utg,test,20,0,high,very_high,none,interested,...,i would not deliver their item if they already...,i would focus even more on G to gain their trust,65,generic_demographics,End,0.0,1,3,3,24
11096,n626q41f,61451e21a303bee08cbb86ea,ta176utg,test,20,0,medium,medium,none,interested,...,I wanted to focus on pleasing G,I wanted to focus on both tasks,65,generic_demographics,End,2.0,3,2,2,60
11097,cppwbnz4,677ecee78287bfab9711dc7b,ta176utg,test,20,0,high,very_high,none,interested,...,"Once Y became angry, I didn't think I could ma...",I wanted to please Y so they would have a good...,65,generic_demographics,End,2.0,3,2,3,34


In [62]:
df_strats = df[['part_idx', 'certainty', 'strategy_high', 'strategy_medium', 'strategy_low']]
# Melt such that focus_work is a single column and value is the value of the focus_work as well as strategy and value is the value of the strategy
df_strategies = df_strats.melt(id_vars=['part_idx', 'certainty'], value_vars=['strategy_high', 'strategy_medium', 'strategy_low'], var_name='trust', value_name='strategy')
df_strategies['trust'] = df_strategies['trust'].str.replace('strategy_', '')

df_strategies = df_strategies.dropna()
# Unstack on trust such that each trust level is a column

df_strategies = df_strategies.set_index(['part_idx', 'trust', 'certainty']).unstack(['trust', 'certainty'])
df_strategies.columns = df_strategies.columns.droplevel(0)



In [68]:
# Pandas to display all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 0)

df_strategies

trust                                                                                                                                                                                                                                                                                                                                                                                 high  \
certainty                                                                                                                                                                                                                                                                                                                                                                        immutable   
part_idx                                                                                                                                                                                                                                                                                                                                                                                     
0          I tended to focus on pleasing my friends until the end of the week, seeing as my boss had such poor memory                                                                                                                                                                                                                                                                        
1          making the choice to upset Y made no difference to the boss                                                                                                                                                                                                                                                                                                                       
2          please V and maintain or improve their view of you                                                                                                                                                                                                                                                                                                                                
3          It was hard to gauge what to do with G as they didn't change their opinion from week to week.                                                                                                                                                                                                                                                                                     
4          If G was already pretty happy, then I wanted to make my boss happy.                                                                                                                                                                                                                                                                                                               
5          Willing to miss a delivery because they would forgive me.                                                                                                                                                                                                                                                                                                                         
6          kind of want to make sure V is still happy                                                                                                                                                                                                                                                                                                                                        
7          If G was going to stay in the same mood no matter what, I might as well maximize the happiness of the boss.                                                              